In [42]:
import pandas as pd

data = pd.read_csv('data/non_annualized_GA_KY.csv')
data.head()

,Unnamed: 0,State,City,Date Sown,Variety,Date of Cut,Julian Day,Yield (tons/acre),Time Since Sown (Days),Time Since Last Harvest (Days),Total Radiation (MJ/m^2),Total Rainfall (mm),Avg Air Temp (C),Avg Min Temp (C),Avg Max Temp (C),Avg Soil Moisture (%),Harvested in Sown Year,First Date of Season,Percent Cover (%)
0,0,Georgia,Athens,10/10/2007,TS 4010,5/6/2008,127,2.02,209,209,922.90,98.288,14.886400,8.691000,21.120200,0.15088,0,1,NaN
1,1,Georgia,Athens,10/10/2007,TS 4010,6/20/2008,172,0.47,254,45,1045.84,85.106,23.029778,16.898889,29.451333,0.12520,0,0,NaN
2,2,Georgia,Athens,10/10/2007,TS 4010,8/28/2008,241,0.10,323,69,953.74,134.634,25.709800,20.641600,31.935200,0.10770,0,0,NaN
3,3,Georgia,Athens,10/10/2007,TS 4010,12/5/2008,340,0.76,422,99,622.84,95.254,9.790200,3.943600,16.041000,0.13408,0,0,90.915344
4,4,Georgia,Athens,10/10/2007,BaraWet 501,5/6/2008,127,1.66,209,209,922.90,98.288,14.886400,8.691000,21.120200,0.15088,0,1,NaN


In [44]:
from sdv.tabular import CTGAN

model = CTGAN()
model.fit(data)

In [45]:
new_data = model.sample(200)

In [46]:
new_data.to_csv('data/non_annualized_synth1_GA_KY.csv')
new_data.head()

,Unnamed: 0,State,City,Date Sown,Variety,Date of Cut,Julian Day,Yield (tons/acre),Time Since Sown (Days),Time Since Last Harvest (Days),Total Radiation (MJ/m^2),Total Rainfall (mm),Avg Air Temp (C),Avg Min Temp (C),Avg Max Temp (C),Avg Soil Moisture (%),Harvested in Sown Year,First Date of Season,Percent Cover (%)
0,1679,Kentucky,Athens,4/5/2016,4030,9/16/2013,268,1.23,1504,27,865.09,81.655808,19.083672,12.960003,30.646631,0.179051,0,0,86.574729
1,1679,Kentucky,Lexington,8/9/2012,Phoenix,5/13/2013,245,0.01,875,27,402.97,41.100000,23.170324,16.326058,16.610695,0.253518,0,0,89.150003
2,1407,Kentucky,Lexington,8/9/2012,Ameristand 403T Plus,4/23/2010,185,0.31,988,206,578.93,55.308057,17.163563,11.358325,27.456524,0.378286,0,0,89.272326
3,1679,Georgia,Lexington,2/21/2008,Saranac AR,7/30/2010,265,0.73,297,210,547.89,104.016367,22.787149,17.134553,22.020828,0.124483,0,0,97.364616
4,0,Georgia,Midville,10/10/2007,CW 35106,9/19/2016,101,0.63,1469,108,527.79,109.933949,21.203060,14.208974,33.536591,0.173393,0,0,NaN


In [47]:
# get aggregate data
#targetDataLoc = 'data/GA_only_063022.csv'
#aggDataLoc = 'data/synth1_GA_only_063022.csv'

aggDf = new_data #pd.read_csv(aggDataLoc)
#aggDf = aggDf.drop("Unnamed: 0",axis=1)
targetDf = data #pd.read_csv(targetDataLoc)
#targetDf = targetDf.drop("Unnamed: 0",axis=1)

In [48]:
############## imports
# general
import statistics
import datetime
#from sklearn.externals import joblib # save and load models
import random
# data manipulation and exploration
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

## machine learning stuff
# preprocessing
from sklearn import preprocessing
# feature selection
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.feature_selection import f_regression
# pipeline
from sklearn.pipeline import Pipeline
# train/testing
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_val_score  
# error calculations
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
# models
from sklearn.linear_model import LinearRegression # linear regression
from sklearn.linear_model import BayesianRidge #bayesisan ridge regression
from sklearn.svm import SVR  # support vector machines regression
from sklearn.gaussian_process import GaussianProcessRegressor # import GaussianProcessRegressor
from sklearn.neighbors import KNeighborsRegressor # k-nearest neightbors for regression
from sklearn.neural_network import MLPRegressor # neural network for regression
from sklearn.tree import DecisionTreeRegressor # decision tree regressor
from sklearn.ensemble import RandomForestRegressor  # random forest regression
from sklearn.ensemble import AdaBoostRegressor # adaboost for regression
# saving models
# from sklearn.externals import joblib
import joblib

# import the API
APILoc = 'API/'

import sys
sys.path.insert(0, APILoc)

from API import *

In [52]:
# filter out the features that will not be used by the machine learning models

# the features to keep:
# xColumnsToKeep = ["Julian Day", "Time Since Sown (Days)", "Time Since Last Harvest (Days)", "Total Radiation (MJ/m^2)",
#                "Total Rainfall (mm)", "Avg Air Temp (C)", "Avg Min Temp (C)", "Avg Max Temp (C)",
#                  "Avg Soil Moisture (%)", "Day Length (hrs)"], "Percent Cover (%)"]

xColumnsToKeep = ["Julian Day", "Time Since Sown (Days)", "Total Radiation (MJ/m^2)",
               "Total Rainfall (mm)", "Avg Air Temp (C)", "Avg Min Temp (C)", "Avg Max Temp (C)",
                 "Avg Soil Moisture (%)"]


#xColumnsToKeep = ["Julian Day", "Time Since Sown (Days)", "Total Radiation (MJ/m^2)", "Total Rainfall (mm)"]

# the target to keep
yColumnsToKeep = ["Yield (tons/acre)"]

# get a dataframe containing the features and the targets
xDf = targetDf[xColumnsToKeep]
test_xDf = targetDf[xColumnsToKeep]
yDf = targetDf[yColumnsToKeep]
test_yDf = targetDf[yColumnsToKeep]

# reset the index
xDf = xDf.reset_index(drop=True)
yDf = yDf.reset_index(drop=True)
test_xDf = test_xDf.reset_index(drop=True)
test_yDf = test_yDf.reset_index(drop=True)

pd.set_option('display.max_rows', 2500)
pd.set_option('display.max_columns', 500)

xCols = list(xDf)

In [53]:
# hide the warnings because training the neural network caues lots of warnings.
import warnings
warnings.filterwarnings('ignore')

# make the parameter grids for sklearn's gridsearchcv
rfParamGrid = {
        'model__n_estimators': [5, 10, 25, 50, 100], # Number of estimators
        'model__max_depth': [5, 10, 15, 20], # Maximum depth of the tree
        'model__criterion': ["mae"]
    }
knnParamGrid ={
        'model__n_neighbors':[2,5,10],
        'model__weights': ['uniform', 'distance'],
        'model__leaf_size': [5, 10, 30, 50]    
    }
svrParamGrid = {
        'model__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
        'model__C': [0.1, 1.0, 5.0, 10.0],
        'model__gamma': ["scale", "auto"],
        'model__degree': [2,3,4,5]
    }
nnParamGrid = {
        'model__hidden_layer_sizes':[(3), (5), (10), (3,3), (5,5), (7,7)],
        'model__solver': ['sgd', 'adam'],
        'model__learning_rate' : ['constant', 'invscaling', 'adaptive'],
        'model__learning_rate_init': [0.1, 0.01, 0.001]      
    }

linRegParamGrid = {}

bayesParamGrid={
        'model__n_iter':[100,300,500]
    }

dtParamGrid = {
    'model__criterion': ['mae'],
    'model__max_depth': [5,10,25,50,100]
    }

aModelList = [(RandomForestRegressor(), rfParamGrid, "rfTup.pkl"),
              (KNeighborsRegressor(), knnParamGrid, "knnTup.pkl"),
              (SVR(), svrParamGrid, "svrTup.pkl"),
             #(MLPRegressor(), nnParamGrid, "nnTup.pkl")],
             (LinearRegression(), linRegParamGrid, "linRegTup.pkl"),
             (BayesianRidge(), bayesParamGrid, "bayesTup.pkl"),
             (DecisionTreeRegressor(), dtParamGrid, "dtTup.pkl")]

N = 10
workingDir = 'working_dir'
numFeatures = 8 # 11

In [54]:
saveMLResults(test_xDf, test_yDf, N, xDf, yDf, aModelList, workingDir, numFeatures, printResults=True)

model:  rfTup
Avg MAE:  195.347
Avg R squared:  0.921
Best MAE:  197.082
Best R squared:  0.929
Parameters of the best model:  {'model__criterion': 'mae', 'model__max_depth': 20, 'model__n_estimators': 100}
Features selected by best model:  ['Julian Day', 'Time Since Sown (Days)', 'Total Radiation (MJ/m^2)', 'Total Rainfall (mm)', 'Avg Air Temp (C)', 'Avg Min Temp (C)', 'Avg Max Temp (C)', 'Avg Soil Moisture (%)']
 
test results on our test data: 
results:
0.0880250195771339
0.9334997385082371
model:  knnTup
Avg MAE:  194.721
Avg R squared:  0.92
Best MAE:  183.389
Best R squared:  0.93
Parameters of the best model:  {'model__leaf_size': 5, 'model__n_neighbors': 10, 'model__weights': 'distance'}
Features selected by best model:  ['Julian Day', 'Time Since Sown (Days)', 'Total Radiation (MJ/m^2)', 'Total Rainfall (mm)', 'Avg Air Temp (C)', 'Avg Min Temp (C)', 'Avg Max Temp (C)', 'Avg Soil Moisture (%)']
 
test results on our test data: 
results:
0.08897337199040409
0.9337171646787661
mo